In [1]:
import ROOT
import numpy as np
from matplotlib import pyplot as plt

from matplotlib.patches import Circle, Wedge, RegularPolygon
from matplotlib.collections import PatchCollection

from AnalysisEvent import AnalysisEvent
import math


In [2]:
ROOT.gSystem.Load("/home/felipe/MG5_aMC_v2_5_5/Delphes/libDelphes")

0

In [3]:
events = AnalysisEvent("/home/felipe/MG5_aMC_v2_5_5/Jet_images_autoencoder/Events/run_01/pp_jj.root")

In [4]:
events.addCollection("jets","Jet")
events.addCollection("genjets","GenJet")
events.addCollection("electrons","Electron")
events.addCollection("muons","Muon")
events.addCollection("particles","Particle")
events.addCollection("towers","Tower")
events.addCollection("tracks","Track")
events.addCollection("eflowtracks","EFlowTrack")
events.addCollection("eflowphotons","EFlowPhoton")
events.addCollection("eflowneutralhads","EFlowNeutralHadron")

In [5]:
class DetectorImage:

    shapescale = 0.3
    
    deta = 0.02
    dphi = 0.02
    
    etamin = -5.0
    etamax = 5.0

    phimin = -math.pi
    phimax = math.pi

    n_bins_eta = int( (etamax-etamin)/deta)
    n_bins_phi = int( (phimax-phimin)/dphi)

    def get_image_from_particles(self,collection,idx):


        img = np.zeros( (self.n_bins_eta, self.n_bins_phi) , np.int16 )

        def find_bin(eta,phi):
            
            #print "Check ",phi," ",self.phimin," ",self.dphi
            etabin = int((eta-self.etamin)/self.deta)
            phibin = int((phi-self.phimin)/self.dphi)

            return (etabin,phibin)
            
        for track in collection:

            (etabin, phibin) = find_bin(track.Eta,track.Phi)

            #print (track.Eta,track.Phi)," ",(etabin, phibin)

            if hasattr(track,"PT"):
                try:
                    img[etabin,phibin] = img[etabin,phibin] + track.PT
                except IndexError:
                    img[0,0] = -999
            else:
                try:
                    img[etabin,phibin] = img[etabin,phibin] + track.ET
                except IndexError:
                    img[0,0] = -999
                    
    def get_image_from_event(self,event,idx):
        xsize = 1
        ysize = 1
        
        #fig = matplotlib.pyplot.figure( figsize=(xsize,ysize) )

        fig, ax = plt.subplots(subplot_kw=dict(xlim=(self.etamin,self.etamax),ylim=(self.phimin,self.phimax)),
                              figsize=(xsize,ysize),dpi=224)        
        patches = []
        ax.axis('off')
        
        for el in event.EFlowTrack:
            coords = ( el.Eta, el.Phi )
            
            #print "Circle at ",coords, math.log(el.PT), " pT ",el.PT
            
            circle = Circle(coords, self.shapescale*math.log(el.PT), fill=False,lw=0.1, color='r')
 
            ax.add_artist(circle)

        for el in event.EFlowPhoton:
            coords = ( el.Eta, el.Phi )
            
            #print "Triangle at ",coords, math.log(el.ET), " ET ",el.ET
            
            square = RegularPolygon(coords, 4, self.shapescale*math.log(el.ET), fill=False,lw=0.1, color='g')
 
            ax.add_artist(square)

        for el in event.EFlowNeutralHadron:
            coords = ( el.Eta, el.Phi )
            
            #print "Pentagon at ",coords, math.log(el.ET), " ET ",el.ET
            
            hexagon = RegularPolygon(coords, 6, self.shapescale*math.log(el.ET), fill=False,lw=0.1, color='b')
 
            ax.add_artist(hexagon)

        extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('colors_ppjj/plotcolors_{}.png'.format(idx),bbox_inches = extent)
        plt.close()

In [6]:
cnt = 0
for event in events:
    cnt +=1
    if (cnt==3500):
        break
    
    dimage = DetectorImage()

    img_photons = dimage.get_image_from_particles(event.EFlowPhoton,cnt)
    img_tracks = dimage.get_image_from_particles(event.EFlowTrack,cnt)
    img_neutrals = dimage.get_image_from_particles(event.EFlowNeutralHadron,cnt)
    
    dimage =  dimage.get_image_from_event(event,cnt)

#    assert( False )


#    print "Event ",cnt

TClass::TClass:0: RuntimeWarning: no dictionary for class CompBase is available
